In [1]:
import pandas as pd
from modules.Date2 import *
from modules.Variables import  *
from modules.Data_Quality import *
import os

In [5]:
end_month = 'Apr'
end_year = '2023/24'

files_info =[]

sheet_name = 'Leeds'
root = './'
input_folder = root+'Input files'
output_folder = root+'Prepared Files/'
combined_folder = root+'Combined files/'
directory = os.fsencode(input_folder)
num = 0

In [6]:
dataframes = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") or filename.endswith(".xls") or filename.endswith(".xlsm"):
        num +=1
        file_path = os.path.join(input_folder, filename)
        # Load the data from the Excel file again, specifying no header to properly identify the row
        data = pd.read_excel(file_path, sheet_name= sheet_name, header=None)
        start_month, start_year, row, start_col = find_starting_point(data)
        end_row, end_col = find_specific_date(data, end_month, end_year)
        dates = convert_date(data, row, start_col, end_col)
        extracted_data = extract_var_columns(data, start_col, end_col)
        extracted_data, region, division, cost_centre = get_metadata(extracted_data, filename, sheet_name)
        final_df = pd.DataFrame({'Date': dates})
        final_df = pd.concat([final_df, extracted_data.reset_index(drop=True)], axis=1)
        num_col = len(final_df.columns)
        has_missing, rows_with_missing = check_missing_values(final_df, exclude_column)
        duplicated = check_duplicates(final_df)
        incorrect_relationship = check_relationships(final_df)
        
        var_name = f"{filename}_missing"
        globals()[var_name] = rows_with_missing
        
        file_info = {'file_name': filename.split('.')[0], 
                     'num': num,
                     'region': region, 
                     'division': division,
                     'cost_centre': cost_centre, 
                     'num_col':num_col, 
                     'Start month':start_month, 
                     'Start year':start_year, 
                     'row':row, 
                     'End column':end_col, 
                     'missing':has_missing, 
                     'duplicated':duplicated, 
                     'incorrect_relationship':incorrect_relationship}
        
        files_info.append(file_info)
        dataframes.append(final_df)
        output_file = filename.split('.')[0]
        output_file = output_file.split('-')[1].strip() +'_' + sheet_name
        prepared_file_path = output_folder + f'{output_file}_ok.csv'
        final_df.to_csv(prepared_file_path, index=False)
files_info = pd.DataFrame(files_info)
files_info.to_csv('files_info.csv', index=False)

In [4]:
combined_df = pd.concat(dataframes, ignore_index=True)
combined_csv_path = combined_folder + 'combined_file_leeds.csv'
combined_df.to_csv(combined_csv_path, index = True)

In [4]:
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") or filename.endswith(".xls") or filename.endswith(".xlsm"):
        num +=1
        file_path = os.path.join(input_folder, filename)
        # Load the data from the Excel file again, specifying no header to properly identify the row
        data = pd.read_excel(file_path, sheet_name= sheet_name, header=None)

In [ ]:
check_missing_values(df, exclude_column)